## Target number of units

In [ ]:
import os 
import os.path
import sys
cwd = os.getcwd()
parentdir =  os.path.abspath(os.path.join(cwd, os.pardir))
sys.path.append(parentdir)

In [ ]:
from sqlalchemy import create_engine
from pysandag.database import get_connection_string
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import utils
import orca
# import datasources
%matplotlib inline

In [ ]:
db_connection_string = get_connection_string('..\data\config.yml', 'mssql_db')
mssql_engine = create_engine(db_connection_string)
scenarios = utils.yaml_to_dict('../data/scenario_config.yaml', 'scenario')
print(scenarios)

# Total capacity 2050

In [ ]:
parcel_capacity_sql = '''
    SELECT sum(capacity_2) AS capacity
      FROM urbansim.urbansim.parcel p'''
capacity_df = pd.read_sql(parcel_capacity_sql ,mssql_engine)
total_capacity = int(capacity_df.values)
print("\n   Total Capacity: {:,}".format(total_capacity))

In [ ]:
# compare to 361,715

# Target housing units 2050

In [ ]:
# Target number of units: units needed
units_needed_sql = '''
    SELECT sum(housing_units_add) as housing_units_to_add
      FROM isam.economic_output.urbansim_housing_units
     WHERE demographic_simulation_id = %s'''
units_needed_sql = units_needed_sql % scenarios['demographic_simulation_id']
units_needed_df =  pd.read_sql(units_needed_sql, mssql_engine)
units_needed = int(units_needed_df.values)
print("\n  Target units to add: {:,} \n       (demographic simulation id {:})".\
      format(units_needed,scenarios['demographic_simulation_id']))

# Remaining Capacity at 2050

In [ ]:
print("\n Remaining capacity 2050: {:,}".format(total_capacity - units_needed))

## Capacity from scheduled developments using sched_dev_do_not_use

In [ ]:
sched_dev_do_not_use_sql = '''
SELECT sum(res_units) as sched_dev_capacity
FROM urbansim.urbansim.scheduled_development_do_not_use'''
site_do_not_use = pd.read_sql(sched_dev_do_not_use_sql,mssql_engine)
site_from_do_not_use = int(site_do_not_use.sched_dev_capacity)

In [ ]:
print("\n  Scheduled developments from scheduled_development_do_not_use: {:,}".format(site_from_do_not_use))

## Capacity from scheduled developments using urbansim.parcel

In [ ]:
# sched dev where capacity > 0
parcel_sched_dev_sql = '''
SELECT sum(capacity_2) AS sched_dev_capacity
      FROM urbansim.urbansim.parcel p 
      WHERE site_id IS NOT NULL'''
parcel_site = pd.read_sql(parcel_sched_dev_sql,mssql_engine)
site_from_parcel = int(parcel_site.values)

In [ ]:
print("\n   Scheduled developments from urbansim.parcel : {:,}".format(int(site_from_parcel)))

# Housing units 2017 urbansim.parcel

In [ ]:
existing_du_sql = '''
  select sum(du) as du_2015,sum(du_2017) as du_2017 
  from [urbansim].[urbansim].[parcel]'''
existing_du_df =  pd.read_sql(existing_du_sql, mssql_engine)
existing_du = int(existing_du_df.du_2017.values)

In [ ]:
print("\n Housing units 2017 from urbansim.parcel: {:,}\n".format(existing_du))

# Target housing units 2017

In [ ]:
housing_unit_sql = '''
    SELECT yr, housing_units1, households, housing_units_add
      FROM isam.economic_output.urbansim_housing_units
     WHERE demographic_simulation_id = %s'''
housing_unit_sql = housing_unit_sql % scenarios['demographic_simulation_id']
hu_df =  pd.read_sql(housing_unit_sql, mssql_engine)
hu_df['total_housing_units_add'] = hu_df.housing_units_add.cumsum()
units_to_add_2017 = int(hu_df.loc[hu_df.yr == 2017].housing_units_add.values[0])
print("\n   Target units to add 2017: {:,}\n".format(units_to_add_2017))

# Housing units end of 2017 - first year simulation 

In [ ]:
du_2017 = existing_du + units_to_add_2017
print("\n Housing units first year of simulation 2017: {:,}\n = {:,} +  {:,} \n".\
      format(du_2017,existing_du,units_to_add_2017))  

# Housing units 2017 from housing unit table

In [ ]:
hu_df_2017 = int(hu_df.loc[hu_df.yr == 2017].housing_units1.values[0])
print("\n Housing units 2017 from isam.economic_output.urbansim_housing_units: {:,}".format(hu_df_2017))

#  <span style="color:red">Difference between database table and first year simulation </span>

In [ ]:
diff_hu_2017 = hu_df_2017 - du_2017 
print("\n Difference between database and calculation for 2017 housing units: {:,}\n".format(diff_hu_2017))

In [ ]:
### Total housing units 2050 : exsiting housing units + housing units needed

In [ ]:
print("\n Housing units total 2050: {:,}\n = {:,} +  {:,} \n".\
      format(hu_df_2017 + units_needed,hu_df_2017,units_needed))  

# Plot (note: increase 2026 and 2027)

In [ ]:
hu_df['capacity'] = total_capacity

In [ ]:
hu_df.head()

In [ ]:
hu_df['remaining_capacity'] = hu_df['capacity'] - hu_df['total_housing_units_add'] 

In [ ]:
fig, axes = plt.subplots(3, 1)


plot_df = hu_df[['yr','housing_units_add']].copy()
plot_df.rename(columns = {'housing_units_add': 'Housing units to add'}, inplace=True)
plot_df.set_index('yr',inplace=True)
plot_df.plot(style='.-',ax=axes.flat[0],figsize=(10,5),color='b')

plot_df = hu_df[['yr','total_housing_units_add','households']].copy()
plot_df.set_index('yr',inplace=True)
plot_df['Total Housing Units'] = existing_du + plot_df['total_housing_units_add']
plot_df.rename(columns = {'households': 'Households'}, inplace=True)
del  plot_df['total_housing_units_add']
plot_df.plot(style='.-',ax=axes.flat[1],figsize=(10,10),color=['darkorange','b'])

plot_df = hu_df[['yr','total_housing_units_add','remaining_capacity']].copy()
plot_df.set_index('yr',inplace=True)
plot_df.rename(columns = {'total_housing_units_add': 'Housing units'}, inplace=True)
plot_df.rename(columns = {'remaining_capacity': 'Remaining capacity'}, inplace=True)
plot_df.plot(style='.-',ax=axes.flat[2],figsize=(10,15),color=['b','chocolate'])
plt.axhline(y=0,color='black', lw=2, alpha=0.5,linestyle='--')
# format
fmt = '{x:,.0f}'
tick = mtick.StrMethodFormatter(fmt)

axes.flat[0].set_xlabel("Simulation Year",size =16)
axes.flat[0].set_ylabel("Housing Units",size =16)
ptitle = 'Housing Units Needed by Year\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[0].set_title(ptitle,size=16) 
axes.flat[0].yaxis.set_major_formatter(tick) 
axes.flat[0].legend(fontsize=14)

# change order of legend in second plot
axes.flat[1].set_xlabel("Simulation Year",size=16)
axes.flat[1].set_ylabel("Total Housing Units",size =16)
ptitle = 'Housing Units and Households \ndemographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[1].set_title(ptitle,size=16) 
axes.flat[1].yaxis.set_major_formatter(tick) 
handles, labels = axes.flat[1].get_legend_handles_labels()
zipped = zip(labels,handles)
zipped.sort(key = lambda t: t[0],reverse=True)
labels, handles = zip(*zipped)
axes.flat[1].legend(handles, labels,fontsize=14)
# axes.flat[1].legend(fontsize=14)

axes.flat[2].set_xlabel("Simulation Year",size =16)
axes.flat[2].set_ylabel("Housing Units (cusum)",size =16)
ptitle = 'Housing Units Needed and Remaining Capacity\n demographic_simulation_id=' + str(scenarios['demographic_simulation_id'])
axes.flat[2].set_title(ptitle,size=16) 
axes.flat[2].yaxis.set_major_formatter(tick) 
axes.flat[2].legend(fontsize=14)

plt.xticks(rotation=25)
plt.tight_layout(pad=0.5, w_pad=0.6, h_pad=2)



plt.show()
fig.savefig('Demographic_simulation_2001.png', format='png', dpi=300)

# Map with folium

In [ ]:
import folium

In [ ]:
m = folium.Map(location=[32.7157,  -117.1611])

In [ ]:
m